In [3]:
import numpy as np
import pandas as pd

In [4]:
df=pd.read_csv('diabetes.csv')

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
# Checking correlation for outcome column in terms of rest of the features
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
# Seperating the traning and testing data
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [8]:
# Scaling all the values
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [9]:
X=scaler.fit_transform(X)

In [10]:
# Train test split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [112]:
# Now we build our model
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [12]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.7896 - accuracy: 0.3632 - val_loss: 0.7507 - val_accuracy: 0.3766
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6874 - accuracy: 0.5261 - val_loss: 0.6763 - val_accuracy: 0.5779
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6216 - accuracy: 0.6857 - val_loss: 0.6265 - val_accuracy: 0.6494
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5745 - accuracy: 0.7345 - val_loss: 0.5942 - val_accuracy: 0.6883
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5421 - accuracy: 0.7704 - val_loss: 0.5684 - val_accuracy: 0.7338
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5183 - accuracy: 0.7736 - val_loss: 0.5522 - val_accuracy: 0.7338
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5007 - accuracy: 0.7834 - val_loss: 0.5395 - val_accuracy: 0.7468
Epoch 8/100


In [14]:
# We build a rough model with our intution, however we can automate this process using hyper-parameter tuning using the keras-tunner library

# We will step by step focus on how to build
# 1. how to select appropriate optimizer
# 2. No of nodes in a layer
# 3. how to select no of layers
# 4. All in all one model

In [18]:
# pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


In [19]:
import keras_tuner as kt

In [20]:
# Here we pass different optimizers and then it will create all the seperate models with each passed optimizer and return those models one by one
def build_model(hp):
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [21]:
# This acts as GridSearch
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [22]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 12s


In [35]:
# To get the best hyper-paramter. As we passed multiple optimizers it will select one of the best and return it
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [39]:
# To ge the best models
model=tuner.get_best_models(num_models=1)[0]

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
# We now train the data using best model obtained by hyperparameter tuning.
# Here initial epoch means , as we have already run 5 epochs above it will start from 6th epoch and go upto 100
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 18ms/step - loss: 0.5443 - accuracy: 0.7329 - val_loss: 0.5403 - val_accuracy: 0.7662
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5288 - accuracy: 0.7427 - val_loss: 0.5298 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5157 - accuracy: 0.7573 - val_loss: 0.5223 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5046 - accuracy: 0.7557 - val_loss: 0.5163 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4955 - accuracy: 0.7590 - val_loss: 0.5118 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4874 - accuracy: 0.7638 - val_loss: 0.5081 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4803 - accuracy: 0.7606 - val_loss: 0.5060 - val_accuracy: 0.7857
Epoch 14

Now we select the best number of neurons on the hidden layer by hyper-parameter tuning

In [44]:
def build_model(hp):
  model=Sequential()

  units=hp.Int('units',min_value=8,max_value=128,step=8) # Here we give lower,upper and step size for incrementation
  # Note: If you dont need to take large steps dont specify the step size which reduces the model complexity

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [50]:
# Here as it creates the json file above we need to store that in the folder hence we store the files in variables called directory and project_name
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='My_proj')

In [51]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 11s


In [52]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [53]:
model=tuner.get_best_models(num_models=1)[0]

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               1080      
                                                                 
 dense_1 (Dense)             (None, 1)                 121       
                                                                 
Total params: 1201 (4.69 KB)
Trainable params: 1201 (4.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 15ms/step - loss: 0.5189 - accuracy: 0.7736 - val_loss: 0.5192 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4895 - accuracy: 0.7704 - val_loss: 0.5053 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4731 - accuracy: 0.7752 - val_loss: 0.5018 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4631 - accuracy: 0.7769 - val_loss: 0.5042 - val_accuracy: 0.7662
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4571 - accuracy: 0.7834 - val_loss: 0.5036 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4507 - accuracy: 0.7785 - val_loss: 0.5038 - val_accuracy: 0.7532
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4472 - accuracy: 0.7818 - val_loss: 0.5052 - val_accuracy: 0.7532
Epoch 14

Selecting number of layers now by tuning the best parameter

In [123]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(120,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
      model.add(Dense(120,activation='relu'))

  model.add(Dense(120,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [124]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='num_layers1')

In [125]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 2 Complete [00h 00m 01s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 03s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
9                 |2                 |num_layers

Epoch 1/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, i

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/py

In [126]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [127]:
tuner.get_best_models(num_models=1)[0]

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for mydir/num_layers1/trial_0/checkpoint

Summing up all the things and finding out the best model in terms of optimizer,Dropouts,number of nodes,number of layers

In [113]:
def build_model(hp):
  model=Sequential()

  counter =0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

      if counter==0:
          # Means we need to form the first layer
          model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                          activation=hp.Choice('activation'+str(i),values=['sigmoid','relu','tanh']),
                          input_dim=8))
          model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      else:
          # Means we are forming some of the middle layers
          model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                          activation=hp.Choice('activation'+str(i),values=['sigmoid','relu','tanh'])))
          model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

      counter=counter+1

# Our last layer
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
              loss='binary_crossentropy',
              metrics=['accuracy'])

  return model

In [117]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='final1')

In [118]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.4610389471054077

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 18s


In [119]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 88,
 'activation0': 'relu',
 'dropout0': 0.2,
 'optimizer': 'nadam',
 'units1': 8,
 'activation1': 'sigmoid',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'sigmoid',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'sigmoid',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'sigmoid',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'sigmoid',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'sigmoid',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'sigmoid',
 'dropout7': 0.1,
 'units8': 8,
 'activation8': 'sigmoid',
 'dropout8': 0.1,
 'units9': 8,
 'activation9': 'sigmoid',
 'dropout9': 0.1}

In [120]:
model=tuner.get_best_models(num_models=1)[0]

In [121]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 88)                792       
                                                                 
 dropout (Dropout)           (None, 88)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 712       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dropout_2 (Dropout)         (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 8)                 7

In [122]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 5s 19ms/step - loss: 0.6817 - accuracy: 0.5847 - val_loss: 0.6721 - val_accuracy: 0.6429
Epoch 7/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6691 - accuracy: 0.6287 - val_loss: 0.6632 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6548 - accuracy: 0.6466 - val_loss: 0.6578 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6569 - accuracy: 0.6482 - val_loss: 0.6544 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6549 - accuracy: 0.6531 - val_loss: 0.6530 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6552 - accuracy: 0.6498 - val_loss: 0.6522 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6548 - accuracy: 0.6450 - val_loss: 0.6520 - val_accuracy: 0.6429
Epoch 13/